In [0]:
# Get Learning Cruves

The goal of this jupyter notebook is to get the learning curves accuracies in tran, validation an test for the baseline model and MLNT-teacher and MLNT-student neural networks for each epoch.

It is intended to be used in google colab.
A folder named *learning_curves* needs to be created in order to store the results.
It retrieves data from the checkpoint folder

SyntaxError: ignored

In [1]:
!pip install wandb

     |████████████████████████████████| 1.4MB 2.7MB/s 
     |████████████████████████████████| 460kB 17.4MB/s 
     |████████████████████████████████| 102kB 9.5MB/s 
     |████████████████████████████████| 112kB 13.4MB/s 
     |████████████████████████████████| 102kB 9.3MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
     |████████████████████████████████| 71kB 6.1MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.2-cp36-none-any.whl size=73605 sha256=6c6ecfbe09bbde909fc6920348fe2dd0b65c9902f76a57011c08d17255f00519
  Stored in directory: /root/.cache/pip/wheels/bc/ed/6c/028dea90d31b359cd2a7c8b0da4db80e41d24a59614154072e
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=2f93a13184c76919b24474f149c31625bd3ec81b2ff75cf867d868bc116cb09d
  Stored in directory: /root/.cache/pip/wheels/ce/0e/7b/58a8a5268655b3ad74feef5aa97946f0addafb3cbb6bd2da23
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=4829

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
import wandb
import sys
sys.path.append('/content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/MLNT_cifar')

# Pytorch libraries
import torch
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

# Internal files
import config
import dataloader
import models
# from baseline import get_model, save_checkpoint

import math
import os
import sys
import time
from collections import OrderedDict
import random

import pandas as pd




noise file noisy_label_kv.txt generated with noise: 0.5


noise file noisy_label_validation.txt generated with noise: 0.5



<IPython.core.display.Javascript object>

wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [0]:
def get_model():

    # Get model from config
    if config.model == "resnet18":
        model = models.resnet18(pretrained=config.pretrained)
    elif config.model == "resnet34":
        model = models.resnet34(pretrained=config.pretrained)
    elif config.model == 'resnet50':
        model = models.resnet50(pretrained=config.pretrained)
    elif config.model == "resnet101":
        model = models.resnet101(pretrained=config.pretrained)
    elif config.model == "resnet152":
        model = models.resnet152(pretrained=config.pretrained)
    elif config.model == "resnext50_32x4d":
        model = models.resnet34(pretrained=config.pretrained)
    elif config.model == 'resnext101_32x8d':
        model = models.resnet50(pretrained=config.pretrained)
    elif config.model == "wide_resnet50_2":
        model = models.resnet101(pretrained=config.pretrained)
    elif config.model == "wide_resnet101_2":
        model = models.resnet152(pretrained=config.pretrained)
    else:
        raise ValueError('%s not supported'.format(config.model))

    # Initialize fc layer
    (in_features, out_features) = model.fc.in_features, model.fc.out_features
    model.fc = torch.nn.Linear(in_features, out_features)
    return model

In [5]:
# Get the original_dataset
loader = dataloader.KeyDataLoader()
train_loader, valid_loader, test_loader = loader.run()

# Hyper Parameter settings
random.seed(config.seed)
# torch.cuda.set_device(config.gpuid)
torch.manual_seed(config.seed)
torch.cuda.manual_seed_all(config.seed)
use_cuda = torch.cuda.is_available()

# Networks setup
print('\nModel setup')
print('| Building network: {}'.format(config.model))
net = get_model()

if use_cuda:
  net.cuda()


Model setup
| Building network: resnet34


/content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/MLNT_cifar/models/resnet.py:122: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight, mode='fan_out')
/content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/MLNT_cifar/models/resnet.py:124: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(m.weight, 1)
/content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/MLNT_cifar/models/resnet.py:125: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(m.bias, 0)


In [0]:
def load_checkpoint(epoch):
  # checkpoint = torch.load(config.drive_dir + '/checkpoint/' + config.checkpoint + "_" + str(epoch) + '.pth.tar')
  # checkpoint = torch.load(config.drive_dir + '/checkpoint/' + config.id + "_student_" + str(epoch) + '.pth.tar')
  checkpoint = torch.load(config.drive_dir + '/checkpoint/' + config.id + "_teacher_" + str(epoch) + '.pth.tar')
  net.load_state_dict(checkpoint['state_dict'])

In [0]:
def test(loader):
    net.eval()
    # test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(loader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        with torch.no_grad():
            outputs = net(inputs).cuda()

        # test_loss += loss.data.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

    # Grab validation results
    test_acc = 100. * correct/total

    return(round(float(test_acc),2))

In [0]:
def get_accy_list(max_epoch, loader):
  max_epoch = 100

  accy_list = []
  for epoch in range(1,101):
    load_checkpoint(epoch)
    accy_list.append(test(loader))
  return(accy_list)
    # print(accy_list)

In [15]:
print("computing test list\n")
test_list  = get_accy_list(config.num_epochs, test_loader)
print(test_list)
print("computing val list\n")
valid_list = get_accy_list(config.num_epochs, valid_loader)
print(valid_list)
print("computing train list\n")
train_list = get_accy_list(config.num_epochs, train_loader)

df = pd.DataFrame()
df["epoch"] = range(1,101)
df["test"] = test_list
df["valid"] = valid_list
df["train"] = train_list

computing test list

[10.0, 19.82, 22.77, 26.07, 28.54, 33.57, 36.68, 39.22, 41.67, 44.74, 46.52, 49.54, 51.77, 53.94, 56.17, 58.59, 61.17, 63.56, 65.37, 67.35, 68.38, 69.43, 70.17, 71.64, 72.93, 74.24, 75.4, 76.2, 77.6, 78.36, 79.34, 79.84, 80.53, 81.05, 81.51, 82.25, 82.59, 82.95, 83.13, 83.32, 83.72, 83.92, 84.1, 84.46, 84.37, 84.34, 84.78, 84.8, 85.12, 85.03, 85.05, 85.21, 85.27, 85.37, 85.74, 85.75, 85.84, 86.09, 85.94, 85.9, 85.85, 85.59, 85.8, 86.15, 86.04, 85.88, 85.92, 85.73, 85.87, 85.93, 86.02, 85.97, 85.77, 85.85, 85.65, 85.61, 85.79, 85.49, 85.52, 85.37, 85.14, 84.67, 84.01, 82.98, 82.05, 81.25, 80.49, 79.49, 78.58, 77.63, 76.6, 75.87, 74.86, 73.83, 73.22, 72.31, 71.51, 70.68, 69.62, 68.96]
computing val list

[9.64, 14.36, 15.44, 17.0, 17.34, 18.46, 20.18, 22.34, 23.88, 24.56, 25.42, 26.86, 27.86, 28.92, 29.78, 30.46, 32.22, 33.52, 33.68, 34.9, 35.48, 35.78, 36.18, 37.02, 37.36, 37.98, 38.5, 38.84, 39.4, 39.56, 40.02, 39.98, 40.44, 40.7, 40.56, 40.66, 40.88, 41.08, 41.1, 

In [16]:
df = pd.DataFrame()
df["epoch"] = range(1,101)
df["test"] = test_list
df["valid"] = valid_list
df["train"] = train_list

df

,epoch,test,valid,train
0,1,10.00,9.64,9.99
1,2,19.82,14.36,14.52
2,3,22.77,15.44,15.74
3,4,26.07,17.00,17.03
4,5,28.54,17.34,17.98
...,...,...,...,...
95,96,72.31,37.20,48.32
96,97,71.51,36.90,48.44
97,98,70.68,36.36,48.81
98,99,69.62,35.90,48.97


In [0]:
# clean_kv = config.drive_dir +"/learning_curves/"+ config.checkpoint+".csv"
# clean_kv = config.drive_dir +"/learning_curves/"+ config.id+"_student.csv"
clean_kv = config.drive_dir +"/learning_curves/"+ config.id+"_teacher.csv"
# clean_kv = "/content/drive/My Drive/Colab_Notebooks/pfm/data/asdf.txt"
df.to_csv(clean_kv, sep=';', index=False,header = True)